In [292]:
import os, sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict
from functools import partial

In [293]:
import dolphindb as ddb 
s = ddb.session()
s.connect("172.16.99.183", 38961, "", "")

True

In [294]:

q1 = 'trade_table = loadTable("dfs://tick_trade", "trades")'
q2 = 'depth_table = loadTable("dfs://tick_depth", "depths")'
s.run(q1)
s.run(q2)

In [295]:
start_date, end_date = "2022-05-15", "2022-05-25"

In [296]:
delay = 15

In [297]:
symbols = {
    "ETHUSDC.UNS": ["ETHUSDT.CUP"]
}

In [298]:
gas_fees = {
    "UNS": 30,
}

In [299]:
starts, ends = [], []
start = datetime.strptime(f"{start_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(f"{end_date} 00:00:00", "%Y-%m-%d %H:%M:%S")
while start < end:
    starts.append(start.strftime("%Y.%m.%dT%H:%M:%S"))
    start += timedelta(days=1)
    ends.append(start.strftime("%Y.%m.%dT%H:%M:%S"))

In [300]:
def get_data_date(sym: str, hedge: str, side: str, delay: int, start: str, end: str, n_levels: int = 20) -> pd.DataFrame:
    t1  = "t1 = select timestamp, sum(volume) as volume, wavg(price, volume) as vwap from trade_table"
    t1 += " where timestamp>={}, timestamp<{}, symbol='{}', side={} group by timestamp order by timestamp".format(
        start, end, sym, side)

    price_type = 'a' if side == '1' else 'b'
    volume_type = price_type + 'v'

    t2  = "t2 = select timestamp, " + price_type + f", {price_type}".join(map(str, list(range(1, n_levels + 1))))
    t2 += ', '
    t2 += volume_type + f", {volume_type}".join(map(str, list(range(1, n_levels + 1))))
    t2 += " from depth_table where timestamp>={}, timestamp<{}, symbol='{}', {}1>0 order by timestamp".format(
        start, end, hedge, price_type)

    t3  = "select * from wj(t1, t2, {}s:{}s, <[".format(delay, delay + 1)
    for level in range(1, n_levels + 1):
        t3 += f"first({price_type}{level}) as {price_type}{level}, first({volume_type}{level}) as {volume_type}{level}, "

    t3  = t3[:-2] + "]>, `timestamp)"

    s.run(t1)
    s.run(t2)
    df = s.run(t3)

    return df


def get_data(sym: str, hedge: str, side: str, delay: int, starts: List, ends: List) -> pd.DataFrame:
    n = len(starts)
    if not n > 0: return pd.DataFrame()

    res = []
    for i in range(n):
        print(f"Loading data {sym} from {starts[i]}")
        df_tmp = get_data_date(sym, hedge, side, delay, starts[i], ends[i])
        if not df_tmp.empty:
            res.append(df_tmp)

    if len(res) > 0:
        return pd.concat(res)

    return pd.DataFrame()

In [301]:
def buy_match(fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        if x["vwap"] < x[f"a{i}"] or tot < 0:
            break

        res += x[f"av{i}"] * x[f"a{i}"]
        pnl += x[f"av{i}"] * (x["vwap"] - x[f"a{i}"])
        tot -= x[f"av{i}"]

    if pnl < fee:
        return 0, 0

    pnl -= fee 

    return res, pnl

def sell_match(fee, n_levels, x):
    res, pnl = 0, 0
    tot = x["volume"]
    for i in range(1, n_levels + 1):
        if x["vwap"] > x[f"b{i}"] or tot < 0:
            break

        res += x[f"bv{i}"] * x[f"b{i}"]
        pnl += x[f"bv{i}"] * (x[f"b{i}"] - x["vwap"])
        tot -= x[f"bv{i}"]

    if pnl < fee:
        return 0, 0

    pnl -= fee

    return res, pnl


In [302]:
results = []
for delay in [15, 20]:
    for sym, hedges in symbols.items():
        _, exch = sym.split('.')
        gas = gas_fees[exch]
        print(f"delay: {delay}, symbol: {sym}")
        for hedge in hedges:
            df_L = get_data(sym, hedge, '1', delay, starts, ends)
            df_S = get_data(sym, hedge, '-1', delay, starts, ends)

            buy_func = partial(buy_match, gas, 20)
            sell_func = partial(sell_match, gas, 20)

            buys = df_L.apply(buy_func, axis=1)
            sells = df_S.apply(sell_func, axis=1)

            df_L["potential_match"] = buys.str[0]
            df_L["potential_pnl"] = buys.str[1]
            df_S["potential_match"] = sells.str[0]
            df_S["potential_pnl"] = sells.str[1]

            df_L["timestamp"] = pd.to_datetime(df_L["timestamp"])
            df_S["timestamp"] = pd.to_datetime(df_S["timestamp"])

            df_L.set_index("timestamp", inplace=True)
            df_S.set_index("timestamp", inplace=True)

            df = pd.DataFrame({
                "LongMatch": df_L["potential_match"].resample("1D").sum(),
                "LongPnL": df_L["potential_pnl"].resample("1D").sum(),
                "ShortMatch": df_S["potential_match"].resample("1D").sum(),
                "ShortPnL": df_S["potential_pnl"].resample("1D").sum(),
            })

            df["symbol"] = sym
            df["hedge"] = hedge
            df["delay"] = delay

            results.append(df)

df = pd.concat(results)

delay: 15, symbol: ETHUSDC.UNS
Loading data ETHUSDC.UNS from 2022.05.15T00:00:00
Loading data ETHUSDC.UNS from 2022.05.16T00:00:00


In [ ]:
df

,LongMatch,LongPnL,ShortMatch,ShortPnL,symbol,hedge,delay
timestamp,,,,,,,
2022-05-15,5.077859e+06,5029.764241,1.632655e+08,155685.233157,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-16,6.631358e+06,9674.467157,1.876869e+08,211654.576032,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-17,3.285923e+06,4906.929510,1.379030e+08,133189.027330,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-18,0.000000e+00,0.000000,0.000000e+00,0.000000,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-19,6.220541e+06,19669.606384,1.716101e+08,171254.741971,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-20,5.168818e+06,5636.098412,1.529105e+08,143012.270026,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-21,5.247573e+05,1123.854844,7.882526e+07,62345.582506,ETHUSDC.UNS,ETHUSDT.CUP,15
2022-05-22,1.203048e+06,1504.573812,8.915242e+07,81702.972868,ETHUSDC.UNS,ETHUSDT.CUP,15


In [ ]:
df_L.to_csv("sample_L.csv", index=True)


In [ ]:
df.to_csv("ETHUSDC.UNS.arb.csv", index=True)